In [2]:
from database.models import Assets, AssetsHistory
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import tools.tools as tools
import requests
import json

### Inicialização do Banco SQLite

In [3]:
assets = Assets()
assets.drop_table()
assets.create_table()

assets_history = AssetsHistory()
assets_history.drop_table()
assets_history.create_table()

### Extração de Dados da API Assets e Inserção no Banco

In [4]:
#Config
coins = "bitcoin,ethereum,dogecoin"

#Extração
BASE_ASSET_URL = f"http://api.coincap.io/v2/assets?ids={coins}"
response = requests.get(BASE_ASSET_URL)
items = response.json()['data']

#Inserção
for item in items:
    assets.insert((item['symbol'], item['name']))

### Extração de Dados da API History e Inserção no Banco

In [5]:
#Configs
today = date.today()
past_date = today - relativedelta(years=5)
dt_format = "%Y-%m-%d"

start_unixtime = tools.to_unixtime_ms(past_date.strftime(dt_format))
end_unixtime = tools.to_unixtime_ms(today.strftime(dt_format))

#Extração
BASE_HISTORY_URL = f'http://api.coincap.io/v2/assets/bitcoin/history?start={start_unixtime}&end={end_unixtime}&interval=d1'
response = requests.get(BASE_HISTORY_URL)

items = response.json()['data']

#Inserção
for item in items:
    assets_history.insert((
        float(item['priceUsd']), 
        item['date'][0:10],
        1))


### Extração de Dados da API para Candles e Inserção no Banco

In [8]:
#Configs
today = date.today()
past_date = today - relativedelta(months=1)
dt_format = "%Y-%m-%d"

start_unixtime = tools.to_unixtime_sec(past_date.strftime(dt_format))
end_unixtime = tools.to_unixtime_sec(today.strftime(dt_format))

#Extração 
BASE_CANDLE_URL = 'https://api.coinmarketcap.com/data-api/v3.1/cryptocurrency/historical'
param = f'?id=1&timeStart={start_unixtime}&timeEnd={end_unixtime}&interval=1d'

print(BASE_CANDLE_URL + param)
response = requests.get(BASE_CANDLE_URL + param)
items = response.json()
print(items)

https://api.coinmarketcap.com/data-api/v3.1/cryptocurrency/historical?id=1&timeStart=1690081200&timeEnd=1692759600&interval=1d
{'data': {'id': 1, 'name': 'Bitcoin', 'symbol': 'BTC', 'timeEnd': '1658275199', 'quotes': [{'timeOpen': '2023-07-24T00:00:00.000Z', 'timeClose': '2023-07-24T23:59:59.999Z', 'timeHigh': '2023-07-24T00:26:00.000Z', 'timeLow': '2023-07-24T15:11:00.000Z', 'quote': {'open': 30081.6616445789, 'high': 30093.3942473004, 'low': 28934.2941602457, 'close': 29176.9154848326, 'volume': 15395817395.41, 'marketCap': 567148527546.04, 'timestamp': '2023-07-24T23:59:59.999Z'}}, {'timeOpen': '2023-07-25T00:00:00.000Z', 'timeClose': '2023-07-25T23:59:59.999Z', 'timeHigh': '2023-07-25T18:41:00.000Z', 'timeLow': '2023-07-25T01:45:00.000Z', 'quote': {'open': 29178.9700445921, 'high': 29353.161104588, 'low': 29062.4342931064, 'close': 29227.390799952, 'volume': 10266772792.7, 'marketCap': 568155078548.47, 'timestamp': '2023-07-25T23:59:59.999Z'}}, {'timeOpen': '2023-07-26T00:00:00.000

### Extração dos Dados Formatados do Banco de Dados

### Gráficos